In [8]:
from operator import add
lines = sc.textFile('fruits.txt')
counts = lines.flatMap(lambda x: x.split()) \
              .map(lambda x: (x, 1)) \
              .reduceByKey(add)\
              .filter(lambda x: len(x[0])>=5)

In [9]:
counts.collect()

[('banana', 1),
 ('canary', 1),
 ('melon', 1),
 ('orange', 1),
 ('pineapple', 1),
 ('apple', 1),
 ('lemon', 1),
 ('strawberry', 1)]

In [7]:
def f(x):
    if x==2:
        return [2]
    else:
        return []
    
rdd = sc.parallelize([2, 3, 4])
sorted(rdd.flatMap(f).collect())

[2]

In [11]:
rdd = sc.parallelize([2, 3, 4])
sorted(rdd.flatMap(lambda x:[]).collect())

[]

In [12]:
A = sc.parallelize(range(1, 100))
t = 50
B = A.filter(lambda x: x < t)
t = 10
C = B.filter(lambda x: x > t)
print(C.count())#0 create channel, use current value of t when performing action

0


In [14]:
A = sc.parallelize(range(1, 100))
t = 50
B = A.filter(lambda x: x < t)
B.cache()#the first time its is computed, the value is stored  (when caching, replace x with current value of 5)
t = 10
C = B.filter(lambda x: x > t)
print(C.count())

39


In [15]:
odd = sc.accumulator(0)
even = 0
def count(element):
	global even
	if element % 2 == 0:
		even += 1
	else:
		odd.add (1)
sc.parallelize([1, 6, 7, 8, 3, 4, 4, 2]).foreach(count)
print(odd, even)

3 0


In [16]:
r1 = sc.parallelize([4, 2, 3, 1, 3, 4, 5, 6, 7, 8, 5], 3)
r2 = sc.parallelize([1, 3, 4, 5, 6, 7, 3, 5, 4, 9, 3], 3)
print(r1.zip(r2).map(lambda x: x[0]*x[1]).reduce(lambda x, y: x+y))

233


In [65]:
r1 = sc.parallelize([4, 2, 3, 1, 3, 4, 5, 6, 7, 8, 5], 3)
r2 = sc.parallelize([1, 3, 4, 5, 6, 7, 3, 5, 4, 9, 3], 3)

# Induce a different partitioning
r2 = r2.flatMap(lambda x: [] if x == 1 else [x])\
       .flatMap(lambda x: [9, 9] if x == 9 else [x])
print(r1.collect())
print(r2.collect())
r1 = r1.zipWithIndex()
r2 = r2.zipWithIndex()
combined = (r1.cartesian(r2))
combined.filter(lambda x: x[0][1] == x[1][1]).map(lambda x:x[0][0]*x[1][0]).sum()


[4, 2, 3, 1, 3, 4, 5, 6, 7, 8, 5]
[3, 4, 5, 6, 7, 3, 5, 4, 9, 9, 3]


273

In [59]:
r1 = r1.zipWithIndex()
r2 = r2.zipWithIndex()
print(r1.collect())
print(r2.collect())

[(4, 0), (2, 1), (3, 2), (1, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (5, 10)]
[(3, 0), (4, 1), (5, 2), (6, 3), (7, 4), (3, 5), (5, 6), (4, 7), (9, 8), (9, 9), (3, 10)]


In [63]:
combined.filter(lambda x: x[0][1] == x[1][1]).map(lambda x:x[0][0]*x[1][0]).sum()

273

#combined = (r1.cartesian(r2))
#combined.collect()


In [34]:
def computeDot(sparseVectorA, sparseVectorB):
    """
    Function to compute dot product of two SparseVectors
    """
    return sparseVectorA.dot(sparseVectorB)

# Use Cartesian function on the RDD to create tuples containing 
# 2-combinations of all the rows in the original RDD

combinationRDD = (r1.cartesian(r2))

# The records in combinationRDD will be of the form 
# [(Index, SV1), (Index, SV1)], therefore, you need to
# filter all the records where the index is not equal giving
# RDD of the form [(Index1, SV1), (Index2, SV2)] and so on,
# then use the map function to use the SparseVector's dot function

dottedRDD = (combinationRDD
             .filter(lambda x: x[0][0] != x[1][0])
             .map(lambda x: computeDot(x[0][1], x[1][1])))
print(dottedRDD.collect())

23/02/25 23:27:11 ERROR Executor: Exception in task 4.0 in stage 31.0 (TID 172)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/tianjiaohe/spark/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/Users/tianjiaohe/spark/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/tianjiaohe/spark/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/tianjiaohe/spark/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*args, **kwargs)
  File "/var/folders/h0/cr6ykp052gngl6yqd4mtpvtm0000gn/T/ipykernel_1805/3699312216.py", line 19, in <lambda>
TypeError: 'int' object is not subscriptable

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 31.0 failed 1 times, most recent failure: Lost task 4.0 in stage 31.0 (TID 172) (tianjiaos-mbp-2 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/tianjiaohe/spark/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/Users/tianjiaohe/spark/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/tianjiaohe/spark/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/tianjiaohe/spark/spark-3.3.1-bin-hadoop2/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*args, **kw